In [2]:
# created new environment for tensorflow sessions
import tensorflow as tf # successful import!

#### Tutorial: https://www.tensorflow.org/get_started/get_started

In [3]:
# Simple computational graph
# Each node takes zero or more tensors as inputs and produces a tensor as an output
# One type of node is a constant - 
# all TensorFlow constants take no input and output a value it stores internally. 
# We can create two floating point Tensors 'node1' and 'node2' as follows:

node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

(<tf.Tensor 'Const:0' shape=() dtype=float32>, <tf.Tensor 'Const_1:0' shape=() dtype=float32>)


In [4]:
# Notice that printing the nodes does not output the values 3.0 and 4.0 as you might expect. 
# Instead, they are nodes that, when evaluated, would produce 3.0 and 4.0, respectively. 
# To actually evaluate the nodes, we must run the computational graph within a session. 
# A session encapsulates the control and state of the TensorFlow runtime.
# The following code creates a Session object and then invokes its run method to run enough 
# of the computational graph to evaluate node1 and node2. 
# By running the computational graph in a session as follows:

sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [5]:
# Combining Tensor nodes with operations (Operations are also nodes). 
# For example, we can add our two constant nodes and produce a new graph as follows:

node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

('node3:', <tf.Tensor 'Add:0' shape=() dtype=float32>)
('sess.run(node3):', 7.0)


In [6]:
# A graph can be parameterized to accept external inputs, known as placeholders. 
# A placeholder is a promise to provide a value later.

a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

In [7]:
# We can evaluate this graph with multiple inputs by using the feed_dict argument 
# to the run method to feed concrete values to the placeholders:

print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


In [8]:
# try adding another node

add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


In [10]:
# In machine learning we will typically want a model that can take arbitrary inputs, 
# such as the one above. 
# To make the model trainable, we need to be able to modify the graph to get new outputs 
# with the same input. 
#Variables allow us to add trainable parameters to a graph. 
# They are constructed with a type and initial value:

W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

In [11]:
# Constants are initialized when you call tf.constant, and their value can never change. 
# By contrast, variables are not initialized when you call tf.Variable. 
# To initialize all the variables in a TensorFlow program, you must explicitly call a 
# special operation as follows:

init = tf.global_variables_initializer()
sess.run(init) 

# init is a handle to the TensorFlow sub-graph that initializes all the global variables. 
# Until we call sess.run, the variables are uninitialized.

In [12]:
# Since x is a placeholder we can evaluate linear_model for several values of x simultaneously: 

print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [14]:
# We've created a model, but we don't know how good it is yet. 
# To evaluate the model on training data we need a y placeholder to provide the desired values
# and we need to write a loss function.
# A loss function measures how far apart the current model is from the provided data. 
# We'll use a standard loss model for linear regression, which sums the squares of the 
# deltas between the current model and the provided data. 
# linear_model - y creates a vector where each element is the corresponding example's 
# error delta. 
# We call tf.square to square that error, then we sum all the squared errors to create 
# a single scalar that abstracts the error of all examples using tf.reduce_sum:

y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})) # this is the loss value

23.66


In [15]:
# We could improve this manually by reassigning the values of W and b to the perfect values 
# of -1 and 1. 
# A variable is initialized to the value provided to tf.Variable but can be changed using 
# operations like tf.assign. 
# For example, W=-1 and b=1 are the optimal parameters for our model. 
# We can change W and b accordingly:

fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})) # now loss value is 0

# We guessed the "perfect" values of W and b, but the whole point of machine learning 
# is to find the correct model parameters automatically. 
# We will show how to accomplish this in the next section.

0.0


#### tf.train API